In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('https://raw.githubusercontent.com/Bintang-Satwika/pemula/main/folder1/ads%20sales.csv')
df

,Unnamed: 0,TV Ad Budget ($),Radio Ad Budget ($),Newspaper Ad Budget ($),Sales ($)
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [2]:
X=df.drop(['Unnamed: 0', 'Sales ($)'], axis=1)
X.head()

,TV Ad Budget ($),Radio Ad Budget ($),Newspaper Ad Budget ($)
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4


In [4]:
mean = np.mean(X, axis=0)
std_dev = np.std(X, axis=0)
X_scaled = (X - mean) / std_dev
X_scaled

,TV Ad Budget ($),Radio Ad Budget ($),Newspaper Ad Budget ($)
0,0.969852,0.981522,1.778945
1,-1.197376,1.082808,0.669579
2,-1.516155,1.528463,1.783549
3,0.052050,1.217855,1.286405
4,0.394182,-0.841614,1.281802
...,...,...,...
195,-1.270941,-1.321031,-0.771217
196,-0.617035,-1.240003,-1.033598
197,0.349810,-0.942899,-1.111852
198,1.594565,1.265121,1.640850


In [4]:
X_train=X_scaled.loc[:159,:]
X_train.shape

(160, 3)

In [5]:
X_test=X_scaled.loc[160:200,:]
X_test.shape

(40, 3)

In [6]:
Y_train=df[['Sales ($)']].loc[:159,:]
Y_train.shape

(160, 1)

In [7]:
Y_test=df[['Sales ($)']].loc[160:200,:]
Y_test.shape

(40, 1)

- Alur Back Propagation<br>
$ \LARGE \frac{dJ}{dW^{[2]}} =\frac{dJ}{dz^{[2]}} . \frac{dz^{[2]}} {dW^{[2]}} \\ $
$ \LARGE \frac{dJ}{dW^{[1]}} =\frac{dJ}{dz^{[1]}} . \frac{dz^{[1]}} {dW^{[1]}} \\ $
$ \LARGE \quad \qquad \frac{dJ}{dz^{[1]}} = \frac{dJ}{dA^{[1]}} * \frac{dA^{[1]}}{dz^{[1]}} \\ $
$\LARGE \qquad \qquad \qquad \frac{dJ}{dA^{[1]}} = \frac{dJ}{dz^{[2]}} .  \frac{dz^{[2]}}{dA^{[1]}}  $

In [8]:
def inisiasi_awal(n):
    np.random.seed(10) 
    W1       = np.random.randn(4, n) *0.01
    b1       =np.zeros((4, 1))
    W2      = np.random.randn(1, 4) *0.01
    b2          =np.zeros((1, 1))
    return W1, W2, b1,b2

def forward_propagation(W1,W2,b1,b2, X, Y_true):
    # layer input ke hidden layer
    z1       = np.dot(W1,X.T)+b1
    z1       =z1.T
    A1=np.where(z1>0, z1, 0)
    assert A1.shape == (Y_true.shape[0], 4), ("shape  A1 salah !: "+str(A1.shape) )
    ################################################
    # hidden layer ke output layer
    z2       = np.dot(W2,A1.T)+b2
    Y_pred       =z2.T
    assert Y_pred.shape ==Y_true.shape, ("shape  Y_prediksi salah !: "+str(Y_pred.shape) )
    return z1, A1, z2, Y_pred

def cost_function(Y_pred, Y_true):
    if not isinstance(Y_true, np.ndarray):
        Y_true=Y_true.to_numpy()
    loss    =   np.square((Y_pred-Y_true))
    cost    = np.mean(loss)
    return cost

def back_propagation(X, W1,z1, A1, W2, z2, Y_pred,Y_true, m):
    # output layer ke hidden layer
    if not isinstance(Y_true, np.ndarray):
        Y_true=Y_true.to_numpy()
    dW2 = (2/m)*np.dot((Y_pred-Y_true).T, A1)
    #print("dW2: ", dW2.shape)
    db2 = np.sum((2/m)*np.dot((Y_pred-Y_true).T, 1), axis=1, keepdims=True)
    #print("db2: " , db2.shape)
    assert dW2.shape ==W2.shape, ("shape  dW2 salah !: "+str(dW2.shape) )
    ###########################################
    # hidden layer ke layer input
    dA1=(2/m)*np.dot((Y_pred-Y_true), W2)
    #print("dA1: ",dA1.shape)
    dz1=dA1*np.where(z1>0, 1, 0)
    #print("dz1: ", dz1.shape)
    dW1=np.dot(dz1.T, X)
    #print("dW1: ", dW1.shape)
    db1=np.sum(np.dot(dz1.T, 1), axis=1, keepdims=True)
    #print("db1: ",db1.shape)
    assert dW1.shape ==W1.shape, ("shape  dW1 salah !: "+str(dW1.shape) )

    return dW1,dW2, db1,db2


def update_parameter(learning_rate, W1, W2, b1, b2 , dW1, dW2, db1, db2):
    W1= W1-learning_rate*dW1
    b1=b1-learning_rate*db1
    ######################
    W2= W2-learning_rate*dW2
    b2=b2-learning_rate*db2
    return W1,W2,b1,b2


### optimisasi Vanilla gradient descent

In [19]:
m, n=X_train.shape
print("jumlah sampel:", m, "dan jumlah Kolom:", n)
W1_vanilla, W2_vanilla, b1_vanilla,b2_vanilla= inisiasi_awal(n)
for i in range(3000):
    z1_vanilla, A1_vanilla,z2_vanilla, Y_pred_vanilla=forward_propagation(W1_vanilla,W2_vanilla,b1_vanilla,b2_vanilla, X_train, Y_train)
    cost_vanilla=cost_function(Y_pred_vanilla,Y_train)
    dW1_vanilla,dW2_vanilla, db1_vanilla,db2_vanilla= back_propagation(X_train,W1_vanilla,z1_vanilla, A1_vanilla,W2_vanilla, z2_vanilla, Y_pred_vanilla, Y_train, m)
    W1_vanilla,W2_vanilla,b1_vanilla,b2_vanilla=update_parameter(0.005, W1_vanilla, W2_vanilla, b1_vanilla, b2_vanilla , dW1_vanilla, dW2_vanilla, db1_vanilla, db2_vanilla)
    print("cost ke-%d: %.4f"%(i+1,cost_vanilla))

jumlah sampel: 160 dan jumlah Kolom: 3
cost ke-1: 224.9796
cost ke-2: 221.0422
cost ke-3: 217.1831
cost ke-4: 213.4007
cost ke-5: 209.6934
cost ke-6: 206.0597
cost ke-7: 202.4981
cost ke-8: 199.0069
cost ke-9: 195.5846
cost ke-10: 192.2298
cost ke-11: 188.9409
cost ke-12: 185.7164
cost ke-13: 182.5547
cost ke-14: 179.4542
cost ke-15: 176.4131
cost ke-16: 173.4297
cost ke-17: 170.5021
cost ke-18: 167.6284
cost ke-19: 164.8063
cost ke-20: 162.0335
cost ke-21: 159.3073
cost ke-22: 156.6247
cost ke-23: 153.9822
cost ke-24: 151.3761
cost ke-25: 148.8017
cost ke-26: 146.2537
cost ke-27: 143.7255
cost ke-28: 141.2102
cost ke-29: 138.6991
cost ke-30: 136.1819
cost ke-31: 133.6467
cost ke-32: 131.0793
cost ke-33: 128.4633
cost ke-34: 125.7792
cost ke-35: 123.0048
cost ke-36: 120.1146
cost ke-37: 117.0798
cost ke-38: 113.8678
cost ke-39: 110.4433
cost ke-40: 106.7691
cost ke-41: 102.8071
cost ke-42: 98.5204
cost ke-43: 93.8763
cost ke-44: 88.8510
cost ke-45: 83.4341
cost ke-46: 77.6345
cost ke-4

In [20]:
_, _,_, Y_test_pred=forward_propagation(W1_vanilla,W2_vanilla,b1_vanilla,b2_vanilla, X_test, Y_test)
cost_test=cost_function(Y_test_pred,Y_test)
print("MSE pada testing:", cost_test)

MSE pada testing: 1.179619061086188


### Menggunakan Repeated K-Fold untuk split train dan test dan jenis optimisasi SGD (stochastic gradient descent)

In [11]:
from sklearn.model_selection import RepeatedKFold

MSE_fold    ={"training":[], "testing":[]}
Weight_fold ={"W1": [], "W2": []}
bias_fold   ={"b1":[], "b2":[]}
index_fold  ={"training":[], "testing":[]}

Y           =df[['Sales ($)']]
rkf         = RepeatedKFold(n_splits=5, n_repeats=2, random_state=10)

for fold, (train_index, test_index) in enumerate(rkf.split(X_scaled, Y)):
    ##################################################################################
    X_train_fold, X_test_fold = X_scaled.loc[train_index], X_scaled.loc[test_index]
    Y_train_fold, Y_test_fold = Y.loc[train_index], Y.loc[test_index]
    n=X_train_fold.shape[1]
    W1_SGD, W2_SGD, b1_SGD,b2_SGD = inisiasi_awal(n)
    ####################################################################################
    if fold ==0:
        print(f"Jumlah sampel Train tiap fold      : {len(train_index)}")
        print(f"Jumlah sampel Test  tiap fold      : {len(test_index)}")  
        print(f"Jumlah atribut input               : {n}\n")

    print(f"Fold {fold+1}:") 
    
    X_train_fold=X_train_fold.to_numpy()
    Y_train_fold=Y_train_fold.to_numpy()
    for epoch in range(300):
        for i in range(len(train_index)):
            X_i=X_train_fold[i].reshape(-1, 3)
            Y_i=Y_train_fold[i].reshape(-1, 1)
            z1_SGD, A1_SGD,z2_SGD, Y_pred_SGD   = forward_propagation(W1_SGD,W2_SGD,b1_SGD,b2_SGD, X_i, Y_i)
            cost_SGD                            = cost_function(Y_pred_SGD,Y_i)
            dW1_SGD,dW2_SGD, db1_SGD, db2_SGD   = back_propagation(X_i,W1_SGD,z1_SGD, A1_SGD,W2_SGD, z2_SGD, Y_pred_SGD, Y_i, len(train_index))
            W1_SGD,W2_SGD,b1_SGD,b2_SGD         = update_parameter(0.01, W1_SGD, W2_SGD, b1_SGD, b2_SGD , dW1_SGD, dW2_SGD, db1_SGD, db2_SGD)
    print("     cost pada training  :", cost_SGD)

    # TESTING VALIDASI
    _, _,_, Y_test_pred     =forward_propagation(W1_SGD,W2_SGD,b1_SGD,b2_SGD, X_test_fold, Y_test_fold)
    cost_test               =cost_function(Y_test_pred,Y_test_fold)
    print("     MSE pada testing    :", cost_test)

    # MENYIMPAN PARAMETER
    index_fold['training'].append(np.squeeze(train_index).tolist())
    index_fold['testing'].append(np.squeeze(test_index).tolist())
    MSE_fold["training"].append(cost_SGD)
    MSE_fold['testing'].append(cost_test)
    Weight_fold['W1'].append(np.squeeze(W1_SGD).tolist())
    Weight_fold['W2'].append(np.squeeze(W2_SGD).tolist())
    bias_fold["b1"].append(b1_SGD)
    bias_fold["b2"].append(b2_SGD)
    

Jumlah sampel Train tiap fold      : 160
Jumlah sampel Test  tiap fold      : 40
Jumlah atribut input               : 3

Fold 1:
     cost pada training  : 1.1364137851516676
     MSE pada testing    : 4.863639224058224
Fold 2:
     cost pada training  : 0.009664183819847712
     MSE pada testing    : 1.0635067658405586
Fold 3:
     cost pada training  : 0.25663653260280855
     MSE pada testing    : 1.284331999806601
Fold 4:
     cost pada training  : 0.09534398673775041
     MSE pada testing    : 1.4885833833759097
Fold 5:
     cost pada training  : 0.20300971286583916
     MSE pada testing    : 1.2643605036926213
Fold 6:
     cost pada training  : 0.1768252825745347
     MSE pada testing    : 1.47788750443157
Fold 7:
     cost pada training  : 0.014402418845586427
     MSE pada testing    : 2.0812203994908542
Fold 8:
     cost pada training  : 0.15555079432456276
     MSE pada testing    : 1.5837504297715141
Fold 9:
     cost pada training  : 0.5200570690661555
     MSE pada testing

In [12]:
fold_terbaik=np.argmin(MSE_fold['testing'])
print("MSE dari testing terkecil pada Fold ke-%d"%(fold_terbaik+1))

MSE dari testing terkecil pada Fold ke-2


In [13]:
# VALIDASI nilai MSE pada Fold sesuai index dari np.argmin()
testing_index=index_fold['testing'][fold_terbaik]
W1_test=np.array(Weight_fold['W1'][fold_terbaik])
W2_test=np.array(Weight_fold['W2'][fold_terbaik])
b1_test=bias_fold['b1'][fold_terbaik]
b2_test=bias_fold['b2'][fold_terbaik]
_, _,_, Y_test_pred     = forward_propagation(W1_test,W2_test,b1_test,b2_test, X_scaled.loc[testing_index], Y.loc[testing_index])
cost_test               = cost_function(Y_test_pred,Y.loc[testing_index])
print("MSE testing pada Fold ke-%d:"%(fold_terbaik+1), cost_test)

MSE testing pada Fold ke-2: 1.0635067658405586
